In [1]:
def Stock_History_Data__(auth, Ticker):
    
    api_key = '**********'
    username = '**********'
    password = '***********'    

    import requests
    import time
    import urllib
    from splinter import Browser

    import json
    import pandas as pd
    from pandas.io.json import json_normalize
    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    
    headers = auth
    ticker = Ticker
    
        # Pulls past OHLCV not including current day's if weekday
    def stock_History():
         
        endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(ticker)

        payload = {'apikey': api_key,
                   'periodType': 'year',
                   'frequencyType' : 'daily',
                   'frequency' : '1',
                   'period' : '5',
                   'needExtendedHourData' : 'false'}

        content = requests.get(url = endpoint, params = payload, headers=headers)
        data = content.json()
        df = pd.DataFrame.from_dict(data, orient='columns')
        df = df['candles']
        df = df.to_json()
        parsed = json.loads(df)
        df = pd.DataFrame.from_dict(parsed, orient='index')
        df['Date'] = pd.to_datetime(df['datetime'], unit='ms').dt.date
        df = df.drop(['datetime'], axis=1)  

        df['Temp_Date'] = df['Date']
        df = df.set_index(['Date'])

        return df
    
        # Pulls today's OHLC 
    def Current_History():
        
        endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/quotes".format(ticker)
        payload = {'apikey': api_key}
        content = requests.get(url = endpoint, params = payload, headers=headers)
        data = content.json()
        dataOpen = pd.to_numeric(pd.Series(data[ticker]['openPrice']), errors='coerce')
        dataHigh = pd.to_numeric(pd.Series(data[ticker]['highPrice']), errors='coerce')
        dataLow = pd.to_numeric(pd.Series(data[ticker]['lowPrice']), errors='coerce')
        dataClose = pd.to_numeric(pd.Series(data[ticker]['lastPrice']), errors='coerce')
        dataDateTime = pd.to_numeric(pd.Series(data[ticker]['quoteTimeInLong']), errors='coerce')


        Cdata = pd.DataFrame()
        Cdata['open'] = dataOpen
        Cdata['high'] = dataHigh
        Cdata['low'] = dataLow
        Cdata['close'] = dataClose
        Cdata['Date'] = pd.to_datetime(dataDateTime, unit='ms').dt.date
        Cdata['Temp_Date'] = Cdata['Date']

        Cdata = Cdata.set_index(['Date'])

        return(Cdata)
    
        # Combines DataFrames
    Frames = [stock_History(), Current_History()]

        # Checks to see if ['Temp_Date'].tail(1) is same for both functions
    if stock_History()['Temp_Date'].tail(1).any() == Current_History()['Temp_Date'].tail(1).any():
        data = stock_History()
    else:
        data = pd.concat(Frames)
        
        # Drops Temp_Date
    data = data.drop(['Temp_Date'], axis=1)
        
        # Changes OHLCV Data to match BackTrader Values.
    data = data.rename(columns={'open':'Open', 'high':'High', 'low':'Low', \
                                'close':'Close', 'volume':'Volume'})
    return(data)    